# Data Cleanup - Tweet Dataset (Darija - Moroccan Dialect)

At this first stage we aim at clean a tweet dataset from Hugging Face (`shmuhammad/AfriSenti-twitter-sentiment`), 
focusing on the Moroccan Darija subset. The goal is to preprocess the text data by removing emojis, usernames,
and applying custom list of Darija stop words. The cleaned data will then be ready for n-gram analysis.

# Part 1: Data Cleaning

In [ ]:
# First of all : install dependencies 
%pip install datasets emoji

In [63]:
# Load the dataset and filter to focus in Moroccan Darija
from datasets import load_dataset
moroccan_tweets = load_dataset("shmuhammad/AfriSenti-twitter-sentiment", 'arq')

In [66]:
print('The dataset format is: ', moroccan_tweets.column_names)
print('The number of elements of train subset: ', len(moroccan_tweets['train']))
print('The number of elements of test subset: ', len(moroccan_tweets['test']))
print('The number of elements of validation subset: ', len(moroccan_tweets['validation']))
print('The labels of the dataset are: ', moroccan_tweets['train'].features['label'].names)
print('')
for i in range(3):
    print(moroccan_tweets['train'][i])


The dataset format is:  {'train': ['tweet', 'label'], 'validation': ['tweet', 'label'], 'test': ['tweet', 'label']}
The number of elements of train subset:  1651
The number of elements of test subset:  958
The number of elements of validation subset:  414
The labels of the dataset are:  ['positive', 'neutral', 'negative']

{'tweet': '@user على حسب موقعك يبدو أنك صاحب نظرة ثاقبة .يخي تبهليل . @user', 'label': 2}
{'tweet': '@user تبهليل هاذا', 'label': 2}
{'tweet': '@user هاذي تبهليل ماشي فهامة', 'label': 2}


In [69]:
# Text Processing (Remove emojis, usernames, URLs, Links, special charcters, punctuation, Remove Latin words, conversion to lower case if applicable)
import re
import emoji
def clean_text(row):
    row['tweet'] = re.sub(r'@\w+', '', row['tweet'])  # Remove usernames
    row['tweet'] = re.sub(r'http\S+|www\S+', '', row['tweet'])  # Remove URLs
    row['tweet'] = re.sub(r'\W+', ' ', row['tweet'])  # Remove special characters and punctuation
    row['tweet'] = re.sub(r'\s+', ' ', row['tweet'])  # Normalize whitespace
    row['tweet'] = re.sub(r'[A-Za-z]+', ' ', row['tweet'])  # Remove Latin words
    row['tweet'] = emoji.replace_emoji(row['tweet'], replace='') # Remove emojis
    row['tweet'] = row['tweet'].strip()
    return row

In [70]:
# Applying Text Processing Function

print('Few examples from the train set')
moroccan_tweets['train'] = moroccan_tweets['train'].map(clean_text)
for i in range(3):
    print(moroccan_tweets['train'][i])
print('')

print('Few examples from the test set')
moroccan_tweets['test'] = moroccan_tweets['test'].map(clean_text)
for i in range(3):
    print(moroccan_tweets['test'][i])
print('')

print('Few examples from the validation set')
moroccan_tweets['validation'] = moroccan_tweets['validation'].map(clean_text)
for i in range(3):
    print(moroccan_tweets['validation'][i])

Few examples from the train set


Map: 100%|██████████| 1651/1651 [00:00<00:00, 7439.61 examples/s]


{'tweet': 'على حسب موقعك يبدو أنك صاحب نظرة ثاقبة يخي تبهليل', 'label': 2}
{'tweet': 'تبهليل هاذا', 'label': 2}
{'tweet': 'هاذي تبهليل ماشي فهامة', 'label': 2}

Few examples from the test set


Map: 100%|██████████| 958/958 [00:00<00:00, 9592.38 examples/s]


{'tweet': 'لاباس الحمدالله يارب العالمين اجمعين  سقسي عليك الخير يارب إسحاق', 'label': 0}
{'tweet': 'لمجرب ولا تسقسي الطبيب', 'label': 1}
{'tweet': 'عليه الميت لوكان يصيب يعيش و ميخليكش و الحي يتكبر و يلعبها من غير الفراق هذاك حاجة متستاهل تبكي عليها كي تروح من لخ', 'label': 1}

Few examples from the validation set


Map: 100%|██████████| 414/414 [00:00<00:00, 8775.18 examples/s]

{'tweet': 'ههههههههههههه شوفي تاريخ بلدك بعدها تحدثي عن تاريخ اسيادك حبيبتي انت غ', 'label': 2}
{'tweet': 'يسموه تبهليل بصح ضحكتني كي ناضت و راحت تجري ل طومبيل ع اساس تم تب', 'label': 2}
{'tweet': 'في علم النفس ساعة ساعة ترفه على روحك بشوية تبهليل', 'label': 0}


In [ ]:
# Stop Words Removal
# We need a list of darija stop words to do this

In [89]:
# Data Validation (Check for empty/null values, remove duplicate entries, text length)
import pandas as pd

# Conversion to pandas Dataframes
moroccan_tweets_train_df = pd.DataFrame(moroccan_tweets['train'])
moroccan_tweets_test_df = pd.DataFrame(moroccan_tweets['test'])
moroccan_tweets_validation_df = pd.DataFrame(moroccan_tweets['validation'])
print(moroccan_tweets_train_df.head())

# Check for empty values
print(moroccan_tweets_train_df.isnull().sum())
print(moroccan_tweets_test_df.isnull().sum())
print(moroccan_tweets_validation_df.isnull().sum())

# remove duplicates
moroccan_tweets_train_df.drop_duplicates()
moroccan_tweets_test_df.drop_duplicates()
moroccan_tweets_validation_df.drop_duplicates()


                                               tweet  label
0  على حسب موقعك يبدو أنك صاحب نظرة ثاقبة يخي تبهليل      2
1                                        تبهليل هاذا      2
2                             هاذي تبهليل ماشي فهامة      2
3  تخاف نجاوب يا ناصر ببلوك لانو طريقة السؤال فيه...      2
4         مرنكة أقسم بالله تبهليل ما بعد منتصف الليل      2
tweet    0
label    0
dtype: int64
tweet    0
label    0
dtype: int64
tweet    0
label    0
dtype: int64


,tweet,label
0,ههههههههههههه شوفي تاريخ بلدك بعدها تحدثي عن ت...,2
1,يسموه تبهليل بصح ضحكتني كي ناضت و راحت تجري ل ...,2
2,في علم النفس ساعة ساعة ترفه على روحك بشوية تبهليل,0
3,باين من قشهم مش عندنا بركا من تبهليل راك كبير ...,2
4,ويفعلوها وجوه البخس مرة اخرى ويقلك وعلاه يكرهو...,2
...,...,...
409,أنا مسامحك ولكن منقدرش ناكل داكشي حلو عليا بزاف,1
410,مات في روحك بزاف,2
411,السلام عليكم وقع لي مشكل كبييير بزاف انا وختي ...,2
412,القنطة كي تكون بزاف تضحك,2


In [90]:
# Longest and shortest tweets on train subset
shortest_tweet = moroccan_tweets_train_df.loc[moroccan_tweets_train_df['tweet'].str.split().str.len().idxmin()]
longest_tweet = moroccan_tweets_train_df.loc[moroccan_tweets_train_df['tweet'].str.split().str.len().idxmax()]
min_len, max_len = len(shortest_tweet['tweet']), len(longest_tweet['tweet'])
print(min_len, max_len)
print(shortest_tweet['tweet'])
print(longest_tweet['tweet'])

4 125
زعما
ايه حتا و أنا قلت هاذي تبقى مسألة شخصية كلها و كيفاش و وقتاش و الطريقة لي تساعدك الزواج بصفة عامة مسألة نسبية و كلها و مكتوبه


# Part 2: N-grams Analysis

In [ ]:
# Tokenization

In [ ]:
# N-gram Generation

In [ ]:
# Frequency Analysis

In [ ]:
# Visualization and Analysis